In [ ]:
# Проверка GPU

!nvidia-smi

import torch

print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():

    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Монтирование Google Drive

from google.colab import drive

drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/LSTM_Trading_Models

In [ ]:
# Клонирование и установка

!git clone https://github.com/askimer/LSTM-Trading-Bot.git

%cd LSTM-Trading-Bot

!pip install -r requirements.txt

In [ ]:
# Запуск пайплайна обработки данных

print("🚀 Запуск пайплайна обработки данных...")



!python get_price_data.py

print("✅ Данные скачаны")



!python clean_data.py

print("✅ Данные очищены")



!python feature_engineer.py

print("✅ Индикаторы добавлены")



print("💾 Данные готовы для обучения")

In [ ]:
# ПОЛНОЦЕННОЕ ОБУЧЕНИЕ LSTM С RandomizedSearchCV

import torch

import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset

from skorch import NeuralNetRegressor

from sklearn.model_selection import RandomizedSearchCV

from skorch.callbacks import EarlyStopping

import numpy as np

import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

import pickle

import time

import os



class LSTMRegressor(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_layers=1):

        super(LSTMRegressor, self).__init__()

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)

        self.linear = nn.Linear(hidden_dim, 1)



    def forward(self, x):

        lstm_out, _ = self.lstm(x)

        predictions = self.linear(lstm_out[:, -1])

        return predictions



def full_train_lstm():

    print("🚀 Начинаем ПОЛНОЦЕННОЕ обучение LSTM с RandomizedSearchCV...")

    print("⏱️ Ожидаемое время: 2-4 часа на GPU")

    start_time = time.time()

    

    # Проверка наличия файла данных

    data_file = 'btc_usdt_training_data/full_btc_usdt_data_feature_engineered.csv'

    if not os.path.exists(data_file):

        print(f"❌ Файл данных не найден: {data_file}")

        print(f"Текущая директория: {os.getcwd()}")

        print("Содержимое текущей директории:")

        try:

            print(os.listdir('.'))

        except:

            print("Не удалось получить список файлов")

        print("Проверяем наличие директории btc_usdt_training_data:")

        if os.path.exists('btc_usdt_training_data'):

            print("Директория существует, содержимое:")

            try:

                print(os.listdir('btc_usdt_training_data'))

            except:

                print("Не удалось получить список файлов в директории")

        else:

            print("Директория btc_usdt_training_data не существует")

        raise FileNotFoundError(f"Файл данных {data_file} не найден. Проверьте выполнение предыдущих шагов пайплайна.")

    

    # Загрузка данных

    df = pd.read_csv(data_file)

    df = df.dropna()

    

    # Удаление константных столбцов

    std_dev = df.std()

    non_constant_columns = std_dev[std_dev != 0].index.tolist()

    df = df[non_constant_columns]

    

    X = df.drop('close', axis=1).values

    y = df['close'].values.reshape(-1, 1)

    

    # Масштабирование

    scaler_X = StandardScaler()

    X = scaler_X.fit_transform(X)

    

    scaler_y = StandardScaler()

    y = scaler_y.fit_transform(y)

    

    # Сохранение скейлеров

    with open('scaler_X.pkl', 'wb') as f:

        pickle.dump(scaler_X, f)

    with open('scaler_y.pkl', 'wb') as f:

        pickle.dump(scaler_y, f)

    

    # Разделение данных

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    

    # Преобразование в тензоры

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"📊 Обучение на устройстве: {device}")

    

    n_features = X_train.shape[1]

    X_train_tensor = torch.tensor(np.array(X_train, copy=True, order='C').reshape(-1, 1, n_features), dtype=torch.float32).to(device)

    y_train_tensor = torch.tensor(np.array(y_train, copy=True, order='C'), dtype=torch.float32).to(device)

    X_test_tensor = torch.tensor(np.array(X_test, copy=True, order='C').reshape(-1, 1, n_features), dtype=torch.float32).to(device)

    y_test_tensor = torch.tensor(np.array(y_test, copy=True, order='C'), dtype=torch.float32).to(device)

    

    # Создание модели для поиска

    net = NeuralNetRegressor(

        LSTMRegressor,

        module__input_dim=n_features,

        max_epochs=50,

        iterator_train__shuffle=True,

        device=device,

        callbacks=[EarlyStopping(patience=5)],

        verbose=0

    )



    # Полные параметры для поиска

    params = {

        'module__hidden_dim': [128, 256, 384, 512, 768, 1024],

        'module__num_layers': [2, 3, 4, 5, 6],

        'lr': [0.02, 0.015, 0.01, 0.005, 0.001],

        'batch_size': [64, 128, 192, 256, 384, 512],

        'max_epochs': [20, 30, 40, 50]

    }



    total_combinations = (len(params['module__hidden_dim']) * len(params['module__num_layers']) * 

                          len(params['lr']) * len(params['batch_size']) * len(params['max_epochs']))

    print(f"🎯 Полный поиск: {total_combinations} возможных комбинаций")

    print("🔍 RandomizedSearchCV: 20 случайных комбинаций × 3-fold CV = 60 обучений")

    

    # RandomizedSearchCV

    random_search = RandomizedSearchCV(

        net,

        params,

        n_iter=20,  # 20 случайных комбинаций

        cv=3,

        scoring='neg_mean_squared_error',

        n_jobs=1,  # Последовательно для стабильности

        random_state=42,

        verbose=2

    )

    

    print("\n🏃 Начинаем обучение...")

    random_search.fit(X_train_tensor, y_train_tensor)

    

    # Результаты

    print("\n" + "="*60)

    print("🏆 РЕЗУЛЬТАТЫ ПОИСКА")

    print("="*60)

    

    best_params = random_search.best_params_

    print("Лучшие параметры:")

    for param, value in best_params.items():

        print(f"  {param}: {value}")

    

    best_score = -random_search.best_score_  # neg_mean_squared_error -> positive

    print(f"\nЛучший MSE: {best_score:.6f}")

    

    # Финальная оценка лучшей модели

    best_model = random_search.best_estimator_

    y_pred = best_model.predict(X_test_tensor)

    final_mse = mean_squared_error(y_test_tensor.cpu(), y_pred.cpu())

    print(f"MSE на тесте: {final_mse:.6f}")

    

    training_time = time.time() - start_time

    print(f"\n⏱️ Общее время обучения: {training_time/3600:.1f} часов")

    

    # Сохранение лучшей модели

    torch.save(best_model, 'best_lstm_model_full.pt')

    

    # Копирование на Google Drive

    !cp best_lstm_model_full.pt /content/drive/MyDrive/LSTM_Trading_Models/

    !cp scaler_X.pkl /content/drive/MyDrive/LSTM_Trading_Models/

    !cp scaler_y.pkl /content/drive/MyDrive/LSTM_Trading_Models/

    

    print("\n💾 Лучшая модель сохранена на Google Drive")

    

    # График результатов поиска

    cv_results = pd.DataFrame(random_search.cv_results_)

    plt.figure(figsize=(12, 8))

    

    # Топ-10 результатов

    top_10 = cv_results.nlargest(10, 'mean_test_score')

    plt.bar(range(len(top_10)), -top_10['mean_test_score'], alpha=0.7)

    plt.title('Топ-10 комбинаций параметров (MSE)')

    plt.xlabel('Комбинация')

    plt.ylabel('MSE')

    plt.grid(True)

    plt.savefig('hyperparameter_search_results.png', dpi=150, bbox_inches='tight')

    plt.show()

    

    !cp hyperparameter_search_results.png /content/drive/MyDrive/LSTM_Trading_Models/

    

    return best_model, final_mse



# Запуск полноценного обучения

best_model, final_mse = full_train_lstm()

In [ ]:
# Финализация

print("\n" + "="*60)

print("🎉 ПОЛНОЦЕННОЕ ОБУЧЕНИЕ ЗАВЕРШЕНО!")

print("="*60)



print("\n📁 СОХРАНЕННЫЕ ФАЙЛЫ:")

print("Google Drive: /MyDrive/LSTM_Trading_Models/")

print("- best_lstm_model_full.pt (лучшая модель)")

print("- scaler_X.pkl, scaler_y.pkl")

print("- hyperparameter_search_results.png")



print("\n💡 ДАЛЬНЕЙШИЕ ШАГИ:")

print("1. Скачайте модель с Google Drive")

print("2. Используйте локально с live_trading.py")

print("3. Для RL обучения запустите следующий notebook")



print("\n🚀 ГОТОВО К РЕАЛЬНОЙ ТОРГОВЛЕ!")

print("="*60)